## Solutions - Problem 3

Get the list of airports in the US from which flights are not departed in the month of 2008 January.

* This is an example for outer join.
* We need to get those airports which are in airport codes but not in 2008 January airlines data set.
* Based on the side of the airport codes data set, we can say left or right. We will be using airport codes as the driving data set and hence we will use left outer join.

Let us start spark context for this Notebook so that we can execute the code provided.

If you want to use terminal for the practice, here is the command to use.

```
spark2-shell \
  --master yarn \
  --name "Joining Data Sets" \
  --conf spark.ui.port=0
```

In [ ]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    appName("Joining Data Sets").
    master("yarn").
    getOrCreate()

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [ ]:
import spark.implicits._

In [ ]:
val airlinesPath = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
val airlines = spark.
    read.
    parquet(airlinesPath)

In [ ]:
airlines.select("Year", "Month", "DayOfMonth", "Origin", "Dest", "CRSDepTime").show

In [ ]:
airlines.count

In [ ]:
val airportCodesPath = "/public/airlines_all/airport-codes"

In [ ]:
def getValidAirportCodes(airportCodesPath: String) = {
    val airportCodes = spark.
        read.
        option("sep", "\t").
        option("header", true).
        option("inferSchema", true).
        csv(airportCodesPath).
        filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country = 'USA'")
    airportCodes
}

In [ ]:
val airportCodes = getValidAirportCodes(airportCodesPath)

In [ ]:
airportCodes.count

In [ ]:
import org.apache.spark.sql.functions.col

In [ ]:
airportCodes.
    join(airlines, col("IATA") === col("Origin"), "left").
    filter("Origin IS NULL").
    select(airportCodes("*"), col("Origin")).
    show